# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

#from l2hmc_eager import gauge_dynamics_eager as gde

In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

#from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

Using TensorFlow backend.


In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [4]:
params = {
    'time_size': 16,
    'space_size': 16,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 5000,
    'save_steps': 1000,
    'logging_steps': 50,
    'annealing_steps': 50,
    'annealing_factor': 0.97,
    'clip_value': 10.,
    'rand': False,
    'metric': 'l2',
    'training_samples_steps': 500,
    'training_samples_length': 100
}

## Create model

In [5]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [ ]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   conv_net=True,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=True,
                   annealing=True,
                   clip_grads=False)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/train_samples.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/annealing_samples.
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/
info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/run_info/
figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_145/figures/
conv_net: True
hmc: False
aux: True
eps_trainable: True
clip_grads: False
annealing: True
beta: 1.0
time_size: 16
space_size: 16
link_type: U1
dim: 2
beta_init: 1.0
beta_final: 8.0
num_samples: 5
num_steps: 5
eps: 0.2
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 0.0001
learning_rate_decay_steps: 100
learning_rate_decay_rate: 0.98
train_steps: 

In [22]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/5000     -2.942e+04      0.1575        0.3648          0.2            1          0.0001    
Saving checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_144/model.ckpt

Time to complete saving: 119.2 

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
Time to complete logging: 187.5 

     1/5000     -3.304e+04     0.000406        0.818        0.1999           1          0.0001    
Time to complete logging: 0.2767

     2/5000 

    75/5000     -2.837e+04     0.0001933         1          0.1948         1.031        0.0001    
    76/5000     -2.656e+04     0.0001944         1          0.1948         1.031        0.0001    
    77/5000     -2.745e+04     0.0001916         1          0.1947         1.031        0.0001    
    78/5000     -2.821e+04     0.0001914         1          0.1946         1.031        0.0001    
    79/5000     -2.565e+04     0.0001913      0.8136        0.1945         1.031        0.0001    
    80/5000     -2.556e+04     0.0001951      0.8322        0.1944         1.031        0.0001    
    81/5000     -3.036e+04     0.0002009         1          0.1943         1.031        0.0001    
    82/5000     -2.939e+04     0.0001934         1          0.1942         1.031        0.0001    
    83/5000     -2.731e+04     0.000194          1          0.1941         1.031        0.0001    
    84/5000     -2.774e+04     0.0001985         1           0.194         1.031        0.0001    
    85/500

   158/5000     -2.832e+04     0.0001933         1          0.1891         1.096        9.8e-05   
   159/5000     -2.697e+04     0.0001932         1           0.189         1.096        9.8e-05   
   160/5000     -2.993e+04     0.0001815         1          0.1889         1.096        9.8e-05   
   161/5000     -2.796e+04     0.0001888       0.907        0.1888         1.096        9.8e-05   
   162/5000     -2.887e+04     0.0001927      0.9961        0.1888         1.096        9.8e-05   
   163/5000     -2.886e+04     0.0001868         1          0.1887         1.096        9.8e-05   
   164/5000     -2.844e+04     0.0001916      0.9886        0.1886         1.096        9.8e-05   
   165/5000     -2.798e+04     0.0001984      0.9309        0.1885         1.096        9.8e-05   
   166/5000     -2.742e+04     0.0002112      0.9429        0.1884         1.096        9.8e-05   
   167/5000     -2.823e+04     0.0002188         1          0.1884         1.096        9.8e-05   
   168/500

   241/5000     -2.709e+04     0.0001844         1          0.1834         1.13        9.604e-05  
   242/5000     -2.674e+04     0.0001895         1          0.1833         1.13        9.604e-05  
   243/5000      -2.79e+04     0.0001852         1          0.1833         1.13        9.604e-05  
   244/5000     -2.752e+04     0.0001886         1          0.1832         1.13        9.604e-05  
   245/5000     -2.748e+04     0.0001912         1          0.1831         1.13        9.604e-05  
   246/5000      -2.88e+04     0.0001907         1          0.1831         1.13        9.604e-05  
   247/5000     -2.794e+04     0.0001902         1           0.183         1.13        9.604e-05  
   248/5000     -2.666e+04     0.0001873         1          0.1829         1.13        9.604e-05  
   249/5000     -2.747e+04     0.0001887         1          0.1828         1.13        9.604e-05  
   250/5000     -2.759e+04     0.0001862      0.8682        0.1828         1.165       9.604e-05  
Time to co

   324/5000     -2.708e+04     0.0001853         1          0.1783         1.201       9.412e-05  
   325/5000     -2.654e+04     0.000187          1          0.1783         1.201       9.412e-05  
   326/5000     -2.552e+04     0.0001831         1          0.1782         1.201       9.412e-05  
   327/5000     -2.697e+04     0.0001932         1          0.1782         1.201       9.412e-05  
   328/5000     -2.855e+04     0.0001884         1          0.1781         1.201       9.412e-05  
   329/5000     -2.841e+04     0.0001884      0.9808        0.1781         1.201       9.412e-05  
   330/5000     -2.858e+04     0.0001876         1           0.178         1.201       9.412e-05  
   331/5000     -2.604e+04     0.0001876         1          0.1779         1.201       9.412e-05  
   332/5000     -2.784e+04     0.0001873         1          0.1779         1.201       9.412e-05  
   333/5000     -2.725e+04     0.0001957      0.9909        0.1778         1.201       9.412e-05  
   334/500

...

  4666/5000     -2.799e+04     0.0001739         1          0.07184          8         3.948e-05  
  4667/5000     -2.761e+04     0.0001775      0.9205        0.07184          8         3.948e-05  
  4668/5000     -2.643e+04     0.0001789       0.938        0.07184          8         3.948e-05  
  4669/5000     -2.747e+04     0.0001776      0.9944        0.07183          8         3.948e-05  
  4670/5000     -2.582e+04     0.0001839      0.9005        0.07183          8         3.948e-05  
  4671/5000     -2.548e+04     0.0001809      0.8499        0.07183          8         3.948e-05  
  4672/5000     -2.696e+04     0.0001767      0.9243        0.07182          8         3.948e-05  
  4673/5000     -2.736e+04     0.0001744      0.9393        0.07182          8         3.948e-05  
  4674/5000     -2.755e+04     0.0001725      0.9763        0.07182          8         3.948e-05  
  4675/5000     -2.747e+04     0.0001748      0.9446        0.07182          8         3.948e-05  
  4676/500

  4749/5000     -2.658e+04     0.0002421         1          0.07162          8         3.869e-05  
  4750/5000     -2.562e+04     0.0002437         1          0.07162          8         3.869e-05  
Time to complete logging: 0.2747

  4751/5000     -2.666e+04     0.0002101      0.9613        0.07162          8         3.869e-05  
  4752/5000     -2.636e+04     0.0002067         1          0.07161          8         3.869e-05  
  4753/5000     -2.595e+04     0.0001978      0.9716        0.07161          8         3.869e-05  
  4754/5000     -2.709e+04     0.000194       0.9909        0.07161          8         3.869e-05  
  4755/5000     -2.658e+04     0.0001895         1          0.07161          8         3.869e-05  
  4756/5000     -2.616e+04     0.0001947      0.7906        0.07161          8         3.869e-05  
  4757/5000     -2.769e+04     0.0001777      0.9912        0.07161          8         3.869e-05  
  4758/5000     -2.466e+04     0.0001855      0.9679        0.07161        

  4832/5000     -2.741e+04     0.000205       0.9717        0.07172          8         3.792e-05  
  4833/5000     -2.646e+04     0.0002076         1          0.07172          8         3.792e-05  
  4834/5000     -2.783e+04     0.0002145      0.9791        0.07173          8         3.792e-05  
  4835/5000     -2.651e+04     0.0002107      0.9228        0.07172          8         3.792e-05  
  4836/5000     -2.833e+04     0.0002159      0.9454        0.07172          8         3.792e-05  
  4837/5000     -2.701e+04     0.0001891      0.8853        0.07172          8         3.792e-05  
  4838/5000     -2.603e+04     0.0001869      0.8432        0.07172          8         3.792e-05  
  4839/5000     -2.786e+04     0.0001862      0.9596        0.07172          8         3.792e-05  
  4840/5000     -2.663e+04     0.0001772      0.9487        0.07172          8         3.792e-05  
  4841/5000     -2.682e+04     0.0001804      0.8491        0.07171          8         3.792e-05  
  4842/500

  4915/5000     -2.774e+04     0.0001812         1          0.07235          8         3.716e-05  
  4916/5000     -2.866e+04     0.0001758         1          0.07234          8         3.716e-05  
  4917/5000     -2.651e+04     0.0001782       0.825        0.07234          8         3.716e-05  
  4918/5000     -2.636e+04     0.0001667         1          0.07234          8         3.716e-05  
  4919/5000     -2.915e+04     0.0001785         1          0.07234          8         3.716e-05  
  4920/5000     -2.654e+04     0.0001775         1          0.07234          8         3.716e-05  
  4921/5000     -2.839e+04     0.0001805      0.9915        0.07234          8         3.716e-05  
  4922/5000     -2.554e+04     0.0001851      0.8214        0.07234          8         3.716e-05  
  4923/5000     -2.871e+04     0.0001784         1          0.07234          8         3.716e-05  
  4924/5000     -2.653e+04      0.00018       0.9267        0.07234          8         3.716e-05  
  4925/500

  4998/5000     -2.839e+04     0.0001919      0.9664        0.07204          8         3.716e-05  
  4999/5000     -2.724e+04     0.0001824      0.9611        0.07203          8         3.716e-05  
--------------------------------------------------------------------------------

Evaluating sampler for 100 steps at beta = 8.0.
Running (trained) L2HMC sampler for 100 steps...
step:      0/100    	 time/step (normalized): 6.067e-05
step:     10/100    	 time/step (normalized): 3.296e-05
step:     20/100    	 time/step (normalized): 3.237e-05
step:     30/100    	 time/step (normalized): 3.204e-05
step:     40/100    	 time/step (normalized): 3.228e-05
step:     50/100    	 time/step (normalized): 3.944e-05
step:     60/100    	 time/step (normalized): 3.541e-05
step:     70/100    	 time/step (normalized): 3.444e-05
step:     80/100    	 time/step (normalized): 3.971e-05
step:     90/100    	 time/step (normalized): 3.457e-05
done.
Samples saved to: /Users/saforem2/ANL/l2hmc/gauge_logs_gr

In [25]:
run_steps = [100, 200, 500, 1000, 5000]#, 10000]

In [26]:
model.beta_final

8.0

In [27]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:      0/100    	 time/step (normalized): 0.0002521
step:     10/100    	 time/step (normalized): 3.193e-05
step:     20/100    	 time/step (normalized): 4.396e-05
step:     30/100    	 time/step (normalized): 3.91e-05
step:     40/100    	 time/step (normalized): 3.684e-05
step:     50/100    	 time/step (normalized): 3.923e-05
step:     60/100    	 time/step (normalized): 3.932e-05
step:     70/100    	 time/step (normalized): 3.797e-05
step:     80/100    	 time/step (normalized): 4.185e-05
step:     90/100    	 time/step (normalized): 4.247e-05
done.
Samples saved to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_144/samples_history/samples_history_100.pkl.
--------------------------------------------------------------------------------

Running (trained) L2HMC sampler for 200 steps...
step:      0/200    	 time/step (normalized): 3.727e-05
step:     10/200    	 time/step (normalized): 3.723e-05
step:     20/200    	 time/step (

step:    530/1000   	 time/step (normalized): 4.178e-05
step:    540/1000   	 time/step (normalized): 3.8e-05
step:    550/1000   	 time/step (normalized): 3.898e-05
step:    560/1000   	 time/step (normalized): 3.897e-05
step:    570/1000   	 time/step (normalized): 3.985e-05
step:    580/1000   	 time/step (normalized): 3.783e-05
step:    590/1000   	 time/step (normalized): 4.058e-05
step:    600/1000   	 time/step (normalized): 3.724e-05
step:    610/1000   	 time/step (normalized): 3.523e-05
step:    620/1000   	 time/step (normalized): 3.53e-05
step:    630/1000   	 time/step (normalized): 3.268e-05
step:    640/1000   	 time/step (normalized): 3.695e-05
step:    650/1000   	 time/step (normalized): 3.594e-05
step:    660/1000   	 time/step (normalized): 4.151e-05
step:    670/1000   	 time/step (normalized): 4.112e-05
step:    680/1000   	 time/step (normalized): 3.618e-05
step:    690/1000   	 time/step (normalized): 3.598e-05
step:    700/1000   	 time/step (normalized): 4.808

step:    960/5000   	 time/step (normalized): 3.316e-05
step:    970/5000   	 time/step (normalized): 3.203e-05
step:    980/5000   	 time/step (normalized): 3.241e-05
step:    990/5000   	 time/step (normalized): 3.243e-05
step:   1000/5000   	 time/step (normalized): 3.172e-05
step:   1010/5000   	 time/step (normalized): 3.23e-05
step:   1020/5000   	 time/step (normalized): 3.319e-05
step:   1030/5000   	 time/step (normalized): 3.223e-05
step:   1040/5000   	 time/step (normalized): 3.191e-05
step:   1050/5000   	 time/step (normalized): 3.75e-05
step:   1060/5000   	 time/step (normalized): 3.308e-05
step:   1070/5000   	 time/step (normalized): 3.217e-05
step:   1080/5000   	 time/step (normalized): 3.568e-05
step:   1090/5000   	 time/step (normalized): 3.391e-05
step:   1100/5000   	 time/step (normalized): 3.378e-05
step:   1110/5000   	 time/step (normalized): 3.525e-05
step:   1120/5000   	 time/step (normalized): 3.766e-05
step:   1130/5000   	 time/step (normalized): 3.31

step:   2430/5000   	 time/step (normalized): 4.097e-05
step:   2440/5000   	 time/step (normalized): 4.044e-05
step:   2450/5000   	 time/step (normalized): 3.819e-05
step:   2460/5000   	 time/step (normalized): 3.721e-05
step:   2470/5000   	 time/step (normalized): 4.649e-05
step:   2480/5000   	 time/step (normalized): 3.508e-05
step:   2490/5000   	 time/step (normalized): 4.742e-05
step:   2500/5000   	 time/step (normalized): 4.297e-05
step:   2510/5000   	 time/step (normalized): 3.481e-05
step:   2520/5000   	 time/step (normalized): 4.403e-05
step:   2530/5000   	 time/step (normalized): 3.933e-05
step:   2540/5000   	 time/step (normalized): 3.918e-05
step:   2550/5000   	 time/step (normalized): 4.326e-05
step:   2560/5000   	 time/step (normalized): 4.879e-05
step:   2570/5000   	 time/step (normalized): 4.131e-05
step:   2580/5000   	 time/step (normalized): 3.676e-05
step:   2590/5000   	 time/step (normalized): 3.454e-05
step:   2600/5000   	 time/step (normalized): 3.

step:   3900/5000   	 time/step (normalized): 3.675e-05
step:   3910/5000   	 time/step (normalized): 3.555e-05
step:   3920/5000   	 time/step (normalized): 4.064e-05
step:   3930/5000   	 time/step (normalized): 4.522e-05
step:   3940/5000   	 time/step (normalized): 3.358e-05
step:   3950/5000   	 time/step (normalized): 3.518e-05
step:   3960/5000   	 time/step (normalized): 3.44e-05
step:   3970/5000   	 time/step (normalized): 3.299e-05
step:   3980/5000   	 time/step (normalized): 3.49e-05
step:   3990/5000   	 time/step (normalized): 3.762e-05
step:   4000/5000   	 time/step (normalized): 3.312e-05
step:   4010/5000   	 time/step (normalized): 3.358e-05
step:   4020/5000   	 time/step (normalized): 3.262e-05
step:   4030/5000   	 time/step (normalized): 3.191e-05
step:   4040/5000   	 time/step (normalized): 3.194e-05
step:   4050/5000   	 time/step (normalized): 3.203e-05
step:   4060/5000   	 time/step (normalized): 3.284e-05
step:   4070/5000   	 time/step (normalized): 3.42

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)